In [ ]:
!pip install -q -r requirements.txt

In [ ]:
import cv2
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
def read_video(video_path):
    video = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))    
    frames = []

    while video.isOpened():
        success, frame = video.read()
        if success:
            frames.append(frame)
        else:
            break

    video.release()
    return frames

In [ ]:
def process_frames(frames, min_detection_confidence=0.2, min_tracking_confidence=0.2):
    mp_drawstyle = mp.solutions.drawing_styles
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    
    # Initialize the list of column names with 'frame'
    columns = ['frame']
    
    # Extend the column list with x, y, z, and v columns for each index i
    for i in range(33):
        columns.extend([f'x{i}', f'y{i}', f'z{i}', f'v{i}'])

    # Create an empty DataFrame with the defined columns
    df = pd.DataFrame(columns=columns)

    with mp_pose.Pose(
        min_detection_confidence=min_detection_confidence,
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        enable_segmentation=False,
        smooth_segmentation=False,
        min_tracking_confidence=min_tracking_confidence
    ) as pose:
        
        # Iterate over each frame and its index
        for i, fr in enumerate(frames):
            # Convert frame color space from BGR to RGB
            fr2 = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)

            # Process frame with pose estimation model
            results = pose.process(fr2)

            # Skip to the next iteration if no pose landmarks are detected
            if not results.pose_landmarks:
                continue

            # Draw landmarks on frame
            mp_drawing.draw_landmarks(fr2, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec=mp_drawstyle.get_default_pose_landmarks_style())
            
            # Store modified frame back into frames list
            frames[i] = fr2
            
            # Initialize row for DataFrame with frame index
            row = [i]

            # Store pose landmark values in the DataFrame
            for landmark in results.pose_landmarks.landmark:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            
            # Append row to the DataFrame
            df.loc[i] = row

    return frames, df

In [ ]:
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)

In [ ]:
def display_frame(frames, index=0):
    plt.imshow(frames[index])
    plt.tight_layout()
    plt.show()

In [ ]:
def collect_data(video_path, output_csv):
    # Read video and get frames
    frames = read_video(video_path)
    
    # Process frames and get DataFrame with pose landmarks
    processed_frames, df = process_frames(frames)
    
    # Save DataFrame to CSV file
    save_to_csv(df, output_csv)
    
    # Display the first frame with pose landmarks
    display_frame(processed_frames)

In [ ]:
collect_data('data/curl-front.mp4', 'coor_curl.csv')

In [ ]:
collect_data('data/lat-raise.mp4', 'coor_lat.csv')

In [ ]:
collect_data('data/shoulder-press.mp4', 'coor_spress.csv')